In [2]:
import pandas as pd
from random import sample
import random
import numpy as np

random.seed(0)
quora_df = pd.read_csv('./data/quora-train.csv', sep="\t")
sts_df = pd.read_csv('./data/sts-train.csv', sep="\t")
quora_df = quora_df.dropna()
sts_df = sts_df.dropna() 
rand_data = sample(range(1, len(quora_df)-1), len(sts_df)-1)
rand_data.insert(0, 0)
quora_df_cut = pd.DataFrame([quora_df.iloc[i] for i in rand_data])

In [3]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)

/home/williamhuang/miniconda3/envs/cs229/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[ 0.02250258 -0.07829181 -0.02303076 ... -0.00827927  0.0265269
  -0.00201898]
 [ 0.04170236  0.0010974  -0.01553419 ... -0.02181627 -0.06359357
  -0.00875284]]


In [7]:
quora_1 = list(quora_df_cut['sentence1'])
quora_2 = list(quora_df_cut['sentence2'])
sts_1 = list(sts_df['sentence1'])
sts_2 = list(sts_df['sentence2'])

In [58]:
sentence_groups = [quora_1, quora_2, sts_1, sts_2]
embeddings = [model.encode(group, show_progress_bar=True) for group in sentence_groups]
quora_embeddings_1 = pd.DataFrame(embeddings[0])
quora_embeddings_2 = pd.DataFrame(embeddings[1])
sts_embeddings_1 = pd.DataFrame(embeddings[2])
sts_embeddings_2 = pd.DataFrame(embeddings[3])
quora_embeddings_1.to_csv("quora_embeddings_1.csv")
quora_embeddings_2.to_csv("quora_embeddings_2.csv")
sts_embeddings_1.to_csv("sts_embeddings_1.csv")
sts_embeddings_2.to_csv("sts_embeddings_2.csv")

Batches: 100%|██████████| 189/189 [00:50<00:00,  3.74it/s]


In [15]:
from torch import nn
import torch
import torch.nn.functional as F

N_PARAPHRASE_CLASSES = 1
N_SIMILARITY_CLASSES= 5
DROPOUT_PROB = 0.5
INPUT_SIZE = 768

class Modelo(nn.Module):
    def __init__(self, model):
        super(Modelo, self).__init__()
        self.model = model
        self.dropout = nn.Dropout(DROPOUT_PROB)
        self.paraphrase_linear = nn.Linear(INPUT_SIZE, INPUT_SIZE // 2)
        self.paraphrase_linear_interact = nn.Linear(INPUT_SIZE, N_PARAPHRASE_CLASSES)
        self.similarity_linear = nn.Linear(INPUT_SIZE, INPUT_SIZE // 2)
        self.similarity_linear_interact = nn.Linear(INPUT_SIZE, N_SIMILARITY_CLASSES)
    
    def forward(self, sentence1, sentence2, task):
        '''
        Task 0 is para. Task 1 is similarity.
        '''
        sentence1 = torch.from_numpy(self.model.encode(sentence1))
        sentence2 = torch.from_numpy(self.model.encode(sentence2))
        if task == 0:
            sentence1 = self.dropout(sentence1)
            sentence1 = F.relu(self.paraphrase_linear(sentence1))
            sentence2 = self.dropout(sentence2)
            sentence2 = F.relu(self.paraphrase_linear(sentence2))
            combined = torch.concat((sentence1, sentence2), dim=-1)
            combined = self.dropout(combined)
            return F.sigmoid(self.paraphrase_linear_interact(combined))
        if task == 1:
            sentence1 = self.dropout(sentence1)
            sentence1 = F.relu(self.similarity_linear(sentence1))
            sentence2 = self.dropout(sentence2)
            sentence2 = F.relu(self.similarity_linear(sentence2))
            combined = torch.concat((sentence1, sentence2), dim=-1)
            combined = self.dropout(combined)
            return F.softmax(self.similarity_linear_interact(combined), dim=-1)


In [ ]:
def save_model(model, optimizer, config, filepath):
    save_info = {
        'model': model.state_dict(),
        'optim': optimizer.state_dict(),
        'model_config': config,
        'system_rng': random.getstate(),
        'numpy_rng': np.random.get_state(),
        'torch_rng': torch.random.get_rng_state(),
    }

    torch.save(save_info, filepath)
    print(f"save the model to {filepath}")

https://huggingface.co/docs/transformers/training#train-in-native-pytorch

In [ ]:
def train_singletask_para_model():
    '''
    use AdamW optimizer.
    binary cross-entropy loss.
    make sure to save model at end to a specific path.
    '''
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    model.train()
    optimizer = AdamW(model.parameters(), lr=5e-5)
    pass 

def train_singletask_sts_model():
    '''
    use AdamW optimizer.
    multi-class cross-entropy loss.
    make sure to save model at end to a specific path.
    '''
    pass

def test_singletask_model():
    '''
    just accuracy for para.
    accuracy + MSE for five class.
    '''
    pass

def train_multitask_model():
    '''
    use AdamW optimizer.
    binary cross-entropy loss for para, multi-class cross-entropy loss for sts, sum loss functions. 
    make sure to save model at end to a specific path.
    '''
    pass 

def test_multitask_model():
    '''
    just accuracy for para.
    accuracy + MSE for five class.
    '''
    pass